In [1]:
# Alpamayo-R1 on YOUR workzone videos (state classification)

import os, sys, re, random, time
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import torch

os.environ["TOKENIZERS_PARALLELISM"] = "false"

ROOT = Path("..").resolve()          # you run from <root>/notebooks
DATA_DIR = ROOT / "data"
VIDEOS_DIR = DATA_DIR / "videos_compressed"

print("ROOT:", ROOT)
print("VIDEOS_DIR exists:", VIDEOS_DIR.exists(), VIDEOS_DIR)


ROOT: /data/wesleyferreiramaia/workzone
VIDEOS_DIR exists: True /data/wesleyferreiramaia/workzone/data/videos_compressed


In [2]:
# If your repo has src/ with alpamayo_r1 inside, keep this:
if os.path.abspath("src") not in sys.path:
    sys.path.append(os.path.abspath("src"))

from alpamayo_r1.models.alpamayo_r1 import AlpamayoR1
from alpamayo_r1.load_physical_aiavdataset import load_physical_aiavdataset
from alpamayo_r1 import helper

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE  = torch.bfloat16 if DEVICE == "cuda" else torch.float16

MODEL_ID = "nvidia/Alpamayo-R1-10B"

print("DEVICE:", DEVICE, "| DTYPE:", DTYPE)

print("Loading model...")
model = AlpamayoR1.from_pretrained(MODEL_ID, dtype=DTYPE).to(DEVICE)
model.eval()

processor = helper.get_processor(model.tokenizer)
print("Model + processor loaded.")

DEVICE: cuda | DTYPE: torch.bfloat16
Loading model...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model + processor loaded.


In [5]:
# Any valid clip_id you already used successfully:
TEMPLATE_CLIP_ID = "030c760c-ae38-49aa-9ad8-f5650a545d26"

print("Loading template data to steal shapes...")
tmpl = load_physical_aiavdataset(TEMPLATE_CLIP_ID, t0_us=5_100_000)

print("Template keys:", tmpl.keys())
print("image_frames:", tmpl["image_frames"].shape, tmpl["image_frames"].dtype)
print("ego_history_xyz:", tmpl["ego_history_xyz"].shape, tmpl["ego_history_xyz"].dtype)
print("ego_history_rot:", tmpl["ego_history_rot"].shape, tmpl["ego_history_rot"].dtype)
print("ego_future_xyz:", tmpl["ego_future_xyz"].shape)

Loading template data to steal shapes...
Template keys: dict_keys(['image_frames', 'camera_indices', 'ego_history_xyz', 'ego_history_rot', 'ego_future_xyz', 'ego_future_rot', 'relative_timestamps', 'absolute_timestamps', 't0_us', 'clip_id'])
image_frames: torch.Size([4, 4, 3, 1080, 1920]) torch.uint8
ego_history_xyz: torch.Size([1, 1, 16, 3]) torch.float32
ego_history_rot: torch.Size([1, 1, 16, 3, 3]) torch.float32
ego_future_xyz: torch.Size([1, 1, 64, 3])


In [6]:
import cv2
import torch
import numpy as np
import re
import textwrap
from pathlib import Path

# --- 1. Robust Text Cleaner ---
def clean_and_wrap_text(raw_text, width=60):
    # Unwrap nested lists recursively (handles [['Text']] or even [[['Text']]])
    while isinstance(raw_text, list):
        if len(raw_text) > 0: raw_text = raw_text[0]
        else: raw_text = ""
            
    if hasattr(raw_text, 'decode'): 
        raw_text = raw_text.decode("utf-8", "ignore")
        
    text = str(raw_text)
    
    # Remove special tokens and prompt echoes
    text = text.replace("<|im_start|>", "").replace("<|im_end|>", "").strip()
    if "assistant" in text:
        text = text.split("assistant")[-1]
        
    text = text.replace("Output the chain-of-thought", "").replace("future trajectory.", "")
    text = text.strip()
    
    # Wrap text into multiple lines for the video
    lines = textwrap.wrap(text, width=width)
    return lines, text

# --- 2. Main Loop ---
def run_final_visualization(video_path, model, processor, tmpl, out_path, stride=3):
    video_path = Path(video_path)
    cap = cv2.VideoCapture(str(video_path))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    
    out = cv2.VideoWriter(str(out_path), cv2.VideoWriter_fourcc(*"mp4v"), fps, (W, H))
    T1, T2, C, tH, tW = tmpl["image_frames"].shape
    
    # "Cruise Control" History (Neutralizing the Ghost Driver)
    cruise_xyz = torch.zeros_like(tmpl["ego_history_xyz"])
    cruise_rot = torch.zeros_like(tmpl["ego_history_rot"])
    cruise_rot[..., 0] = 1.0
    for i in range(cruise_xyz.shape[1]):
        cruise_xyz[0, i, 0] = 10.0 * ((i - 20) * 0.1)

    print(f"Processing: {video_path.name}")

    try:
        while True:
            frames_bgr = []
            sampled_rgb = []
            for i in range(int(T1*T2) * stride):
                ret, frame = cap.read()
                if not ret: break
                frames_bgr.append(frame)
                if i % stride == 0:
                    fr = cv2.resize(frame, (tW, tH))
                    fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
                    sampled_rgb.append(fr)
            
            if len(sampled_rgb) != int(T1*T2): break 

            tensor = torch.from_numpy(np.stack(sampled_rgb))
            tensor = tensor.permute(0, 3, 1, 2).unsqueeze(0)
            
            # --- Native Prompt ---
            messages = helper.create_message(tensor[0])
            instruction_text = "Output the chain-of-thought reasoning of the driving process, then output the future trajectory."
            
            if isinstance(messages[0]["content"], list):
                messages[0]["content"].append({"type": "text", "text": instruction_text})
            else:
                messages[0]["content"] = instruction_text

            inputs = processor.apply_chat_template(
                messages, 
                tokenize=True, 
                add_generation_prompt=True,
                return_dict=True, 
                return_tensors="pt",
            )
            
            model_inputs = {
                "tokenized_data": inputs,
                "ego_history_xyz": cruise_xyz, 
                "ego_history_rot": cruise_rot,
            }
            model_inputs = helper.to_device(model_inputs, "cuda")

            # Inference
            with torch.autocast("cuda", dtype=torch.bfloat16):
                _, _, extra = model.sample_trajectories_from_data_with_vlm_rollout(
                    data=model_inputs,
                    top_p=0.8, 
                    temperature=0.6, 
                    num_traj_samples=1,
                    max_generation_length=256,
                    return_extra=True,
                )

            # --- Clean Data for Display ---
            raw_output = extra.get("cot", [""])[0]
            lines, full_text = clean_and_wrap_text(raw_output, width=65)
            
            # Print FULL text to console (No cutting!)
            print(f"Thought: {full_text}")

            # Draw on Video
            for fr in frames_bgr:
                # Black banner at the top
                banner_height = 50 + (len(lines) * 35)
                cv2.rectangle(fr, (0, 0), (W, banner_height), (0, 0, 0), -1)
                
                # Title
                cv2.putText(fr, "ALPAYMAO REASONING:", (20, 35), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                
                # Draw lines
                y = 80
                for line in lines:
                    cv2.putText(fr, line, (20, y), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                    y += 35
                
                out.write(fr)

    finally:
        cap.release()
        out.release()
        print(f"Saved: {out_path}")

run_final_visualization(
    video_path=Path("/home/wesleyferreiramaia/data/workzone/data/demo/san_antonio.mp4"),
    model=model,
    processor=processor,
    tmpl=tmpl,
    out_path=Path("alpamayo_result.mp4"),
)

Processing: san_antonio.mp4
Thought: [['Keep lane to continue driving since the lane ahead is clear.']]
Thought: [['Keep lane to continue driving since no critical agent needs attention.']]
Thought: [['Slow down to stop at the stop sign ahead.']]
Thought: [['Stop for the stop sign at the intersection.']]
Thought: [['Nudge to the left to increase clearance from the construction equipment on the right.']]
Thought: [['Nudge to the left to increase clearance from the construction worker on the right.']]
Thought: [['Nudge left to increase clearance from the construction worker on the right.']]
Thought: [['Nudge to the left to increase clearance from the construction cones intruding into the lane.']]
Thought: [['Nudge to the left to avoid the cone in the same lane.']]
Saved: alpamayo_result.mp4


In [14]:
import time
import cv2
import torch
import numpy as np
import re
import textwrap
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, Image, clear_output

# --- Configuration ---
VIDEO_DIR = Path("/home/wesleyferreiramaia/data/workzone/data/demo/")

# --- 1. Helper: Text Cleaning ---
def clean_and_wrap_text(raw_text, width=65):
    while isinstance(raw_text, list):
        if len(raw_text) > 0: raw_text = raw_text[0]
        else: raw_text = ""
            
    if hasattr(raw_text, 'decode'): 
        raw_text = raw_text.decode("utf-8", "ignore")
        
    text = str(raw_text)
    text = text.replace("<|im_start|>", "").replace("<|im_end|>", "").strip()
    if "assistant" in text:
        text = text.split("assistant")[-1]
        
    text = text.replace("Output the chain-of-thought", "").replace("future trajectory.", "")
    text = text.strip()
    
    lines = textwrap.wrap(text, width=width)
    return lines, text

# --- 2. Inference Function (Optimized) ---
def run_live_inference(video_path, model, processor, tmpl, out_path, stride=3):
    video_path = Path(video_path)
    cap = cv2.VideoCapture(str(video_path))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_video = cap.get(cv2.CAP_PROP_FPS) or 30.0
    
    out = cv2.VideoWriter(str(out_path), cv2.VideoWriter_fourcc(*"mp4v"), fps_video, (W, H))
    T1, T2, C, tH, tW = tmpl["image_frames"].shape
    
    # "Cruising" History Fix (Neutralizes Ghost Driver)
    cruise_xyz = torch.zeros_like(tmpl["ego_history_xyz"])
    cruise_rot = torch.zeros_like(tmpl["ego_history_rot"])
    cruise_rot[..., 0] = 1.0
    for i in range(cruise_xyz.shape[1]):
        cruise_xyz[0, i, 0] = 10.0 * ((i - 20) * 0.1)

    print(f"Starting Live Inference on: {video_path.name}")

    try:
        while True:
            t_start = time.time() # Start Timer
            
            frames_bgr = []
            sampled_rgb = []
            for i in range(int(T1*T2) * stride):
                ret, frame = cap.read()
                if not ret: break
                frames_bgr.append(frame)
                if i % stride == 0:
                    fr = cv2.resize(frame, (tW, tH))
                    fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
                    sampled_rgb.append(fr)
            
            if len(sampled_rgb) != int(T1*T2): break 

            tensor = torch.from_numpy(np.stack(sampled_rgb))
            tensor = tensor.permute(0, 3, 1, 2).unsqueeze(0)
            
            # Prepare Prompt
            messages = helper.create_message(tensor[0])
            instruction_text = "Output the chain-of-thought reasoning of the driving process, then output the future trajectory."
            
            if isinstance(messages[0]["content"], list):
                messages[0]["content"].append({"type": "text", "text": instruction_text})
            else:
                messages[0]["content"] = instruction_text

            inputs = processor.apply_chat_template(
                messages, 
                tokenize=True, 
                add_generation_prompt=True,
                return_dict=True, 
                return_tensors="pt",
            )
            
            model_inputs = {
                "tokenized_data": inputs,
                "ego_history_xyz": cruise_xyz, 
                "ego_history_rot": cruise_rot,
            }
            model_inputs = helper.to_device(model_inputs, "cuda")

            # Inference Call
            with torch.autocast("cuda", dtype=torch.bfloat16):
                _, _, extra = model.sample_trajectories_from_data_with_vlm_rollout(
                    data=model_inputs,
                    top_p=0.8, 
                    temperature=0.6, 
                    num_traj_samples=1,
                    # OPTIMIZATION: Reduced from 256 to 64 for speed
                    max_generation_length=64,  
                    return_extra=True,
                )

            # Process Output
            raw_output = extra.get("cot", [""])[0]
            lines, full_text = clean_and_wrap_text(raw_output, width=65)
            
            # Calculate FPS
            t_end = time.time()
            fps_val = 1.0 / (t_end - t_start)

            # Draw on Video Frames
            for fr in frames_bgr:
                # Black Banner
                banner_height = 60 + (len(lines) * 35)
                cv2.rectangle(fr, (0, 0), (W, banner_height), (0, 0, 0), -1)
                
                # Title & Speed
                cv2.putText(fr, f"ALPAYMAO REASONING ({fps_val:.2f} FPS):", (20, 35), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                
                # Reasoning Text
                y = 80
                for line in lines:
                    cv2.putText(fr, line, (20, y), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                    y += 35
                
                out.write(fr)

            # Live Display Update
            _, encoded_img = cv2.imencode('.jpg', frames_bgr[-1]) 
            clear_output(wait=True)
            display(Image(data=encoded_img))
            print(f"⚡ Speed: {fps_val:.2f} FPS | Video: {video_path.name}")
            print(f"Thought: {full_text}")

    finally:
        cap.release()
        out.release()
        print(f"\n✅ Video saved to: {out_path}")

# --- 3. Interactive Dashboard ---
def launch_interactive_dashboard():
    # Scan for videos
    video_files = sorted([f.name for f in VIDEO_DIR.glob("*.mp4")])
    
    if not video_files:
        print(f"No MP4 videos found in {VIDEO_DIR}")
        return

    # Create Widgets
    dropdown = widgets.Dropdown(
        options=video_files,
        description='Video:',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    
    btn_run = widgets.Button(
        description='▶ Run Model',
        button_style='success',
        icon='play',
        layout=widgets.Layout(width='20%')
    )
    
    out_log = widgets.Output()

    # Button Action
    def on_button_click(b):
        with out_log:
            clear_output() 
            
            selected_video = VIDEO_DIR / dropdown.value
            
            # Dynamic Naming: my_video.mp4 -> my_video_alpamayo.mp4
            new_name = f"{selected_video.stem}_alpamayo.mp4"
            output_path = Path(new_name)
            
            print(f"🚀 Loading: {selected_video.name}...")
            print(f"💾 Output will be saved to: {new_name}")
            
            try:
                run_live_inference(
                    video_path=selected_video,
                    model=model,
                    processor=processor,
                    tmpl=tmpl,
                    out_path=output_path,
                    stride=3
                )
                print(f"\n✅ Finished! Saved to {new_name}")
            except Exception as e:
                print(f"\n❌ Error: {e}")

    btn_run.on_click(on_button_click)

    # Display
    print(f"📂 Found {len(video_files)} videos in {VIDEO_DIR.name}")
    display(widgets.HBox([dropdown, btn_run]))
    display(out_log)

# --- Start ---
launch_interactive_dashboard()

📂 Found 16 videos in demo


Output()

In [ ]:
import time
import cv2
import torch
import numpy as np
import re
import textwrap
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, Image, clear_output

# --- Configuration ---
VIDEO_DIR = Path("/home/wesleyferreiramaia/data/workzone/data/demo/")

def clean_and_wrap_text(raw_text, width=65):
    while isinstance(raw_text, list):
        if len(raw_text) > 0: raw_text = raw_text[0]
        else: raw_text = ""
    if hasattr(raw_text, 'decode'): 
        raw_text = raw_text.decode("utf-8", "ignore")
    text = str(raw_text)
    text = text.replace("<|im_start|>", "").replace("<|im_end|>", "").strip()
    if "assistant" in text:
        text = text.split("assistant")[-1]
    text = text.replace("Output the chain-of-thought", "").replace("future trajectory.", "")
    text = text.strip()
    lines = textwrap.wrap(text, width=width)
    return lines, text

def run_live_inference(video_path, model, processor, tmpl, out_path, stride=10): # <--- Default stride increased
    video_path = Path(video_path)
    cap = cv2.VideoCapture(str(video_path))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_video = cap.get(cv2.CAP_PROP_FPS) or 30.0
    
    out = cv2.VideoWriter(str(out_path), cv2.VideoWriter_fourcc(*"mp4v"), fps_video, (W, H))
    T1, T2, C, tH, tW = tmpl["image_frames"].shape
    
    # Cruising History
    cruise_xyz = torch.zeros_like(tmpl["ego_history_xyz"])
    cruise_rot = torch.zeros_like(tmpl["ego_history_rot"])
    cruise_rot[..., 0] = 1.0
    for i in range(cruise_xyz.shape[1]):
        cruise_xyz[0, i, 0] = 10.0 * ((i - 20) * 0.1)

    print(f"Starting High-Speed Inference on: {video_path.name}")

    try:
        while True:
            t_start = time.time()
            
            frames_bgr = []
            sampled_rgb = []
            
            # --- SPEED HACK: Process bigger chunks of video at once ---
            # Stride determines how many frames we read per loop.
            # Higher stride = More frames processed per model call = Higher FPS
            frames_to_read = int(T1*T2) * stride
            
            for i in range(frames_to_read):
                ret, frame = cap.read()
                if not ret: break
                frames_bgr.append(frame)
                
                # We still only sample a few frames for the model to see
                if i % stride == 0 and len(sampled_rgb) < int(T1*T2):
                    fr = cv2.resize(frame, (tW, tH))
                    fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
                    sampled_rgb.append(fr)
            
            if len(frames_bgr) == 0: break 

            # Pad if we ran out of frames early
            while len(sampled_rgb) < int(T1*T2):
                sampled_rgb.append(sampled_rgb[-1])

            tensor = torch.from_numpy(np.stack(sampled_rgb))
            tensor = tensor.permute(0, 3, 1, 2).unsqueeze(0)
            
            # Prompt
            messages = helper.create_message(tensor[0])
            instruction_text = "Output the chain-of-thought reasoning of the driving process, then output the future trajectory."
            if isinstance(messages[0]["content"], list):
                messages[0]["content"].append({"type": "text", "text": instruction_text})
            else:
                messages[0]["content"] = instruction_text

            inputs = processor.apply_chat_template(
                messages, tokenize=True, add_generation_prompt=True,
                return_dict=True, return_tensors="pt",
            )
            
            model_inputs = {
                "tokenized_data": inputs,
                "ego_history_xyz": cruise_xyz, 
                "ego_history_rot": cruise_rot,
            }
            model_inputs = helper.to_device(model_inputs, "cuda")

            with torch.autocast("cuda", dtype=torch.bfloat16):
                _, _, extra = model.sample_trajectories_from_data_with_vlm_rollout(
                    data=model_inputs,
                    top_p=0.8, 
                    temperature=0.6, 
                    num_traj_samples=1,
                    max_generation_length=40,  # <--- SPEED HACK: Extremely short reasoning (40 tokens)
                    return_extra=True,
                )

            # Process Output
            raw_output = extra.get("cot", [""])[0]
            lines, full_text = clean_and_wrap_text(raw_output, width=65)
            
            # Draw on ALL frames in the batch (Visual Persistence)
            for fr in frames_bgr:
                banner_height = 60 + (len(lines) * 35)
                cv2.rectangle(fr, (0, 0), (W, banner_height), (0, 0, 0), -1)
                
                t_now = time.time()
                current_fps = len(frames_bgr) / (t_now - t_start + 1e-5)
                
                cv2.putText(fr, f"ALPAMAYO ({current_fps:.1f} FPS):", (20, 35), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                
                y = 80
                for line in lines:
                    cv2.putText(fr, line, (20, y), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                    y += 35
                out.write(fr)

            # Live Display
            _, encoded_img = cv2.imencode('.jpg', frames_bgr[-1]) 
            clear_output(wait=True)
            display(Image(data=encoded_img))
            
            # Print Stats
            fps_val = len(frames_bgr) / (time.time() - t_start)
            print(f"⚡ Speed: {fps_val:.2f} FPS (Batch: {len(frames_bgr)} frames)")
            print(f"Thought: {full_text}")

    finally:
        cap.release()
        out.release()
        print(f"\n✅ Video saved to: {out_path}")

def launch_interactive_dashboard():
    video_files = sorted([f.name for f in VIDEO_DIR.glob("*.mp4")])
    if not video_files: return

    dropdown = widgets.Dropdown(options=video_files, description='Video:', layout=widgets.Layout(width='60%'))
    
    # Slider for Speed Control
    slider_stride = widgets.IntSlider(
        value=15, min=3, max=30, step=1, 
        description='Speed (Stride):', 
        style={'description_width': 'initial'}
    )
    
    btn_run = widgets.Button(description='▶ Run', button_style='success', icon='play')
    out_log = widgets.Output()

    def on_button_click(b):
        with out_log:
            clear_output()
            selected_video = VIDEO_DIR / dropdown.value
            new_name = f"{selected_video.stem}_alpamayo_fast.mp4"
            
            print(f"🚀 Loading with Stride {slider_stride.value}...")
            run_live_inference(
                video_path=selected_video,
                model=model,
                processor=processor,
                tmpl=tmpl,
                out_path=Path(new_name),
                stride=slider_stride.value # <--- Use the slider value
            )

    btn_run.on_click(on_button_click)
    display(widgets.VBox([dropdown, slider_stride, btn_run, out_log]))

launch_interactive_dashboard()

In [ ]:
import time
import cv2
import torch
import numpy as np
import re
import textwrap
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, Image, clear_output

# --- Configuration ---
VIDEO_DIR = Path("/home/wesleyferreiramaia/data/workzone/data/demo/")

# --- Helper Functions ---
def clean_and_wrap_text(raw_text, width=65):
    while isinstance(raw_text, list):
        if len(raw_text) > 0: raw_text = raw_text[0]
        else: raw_text = ""
    if hasattr(raw_text, 'decode'): 
        raw_text = raw_text.decode("utf-8", "ignore")
    text = str(raw_text)
    text = text.replace("<|im_start|>", "").replace("<|im_end|>", "").strip()
    if "assistant" in text:
        text = text.split("assistant")[-1]
    text = text.replace("Output the chain-of-thought", "").replace("future trajectory.", "")
    text = text.strip()
    lines = textwrap.wrap(text, width=width)
    return lines, text

def run_async_simulation(video_path, model, processor, tmpl, out_path, reasoning_interval=10):
    video_path = Path(video_path)
    cap = cv2.VideoCapture(str(video_path))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_video = cap.get(cv2.CAP_PROP_FPS) or 30.0
    
    out = cv2.VideoWriter(str(out_path), cv2.VideoWriter_fourcc(*"mp4v"), fps_video, (W, H))
    T1, T2, C, tH, tW = tmpl["image_frames"].shape
    
    # Cruising History
    cruise_xyz = torch.zeros_like(tmpl["ego_history_xyz"])
    cruise_rot = torch.zeros_like(tmpl["ego_history_rot"])
    cruise_rot[..., 0] = 1.0
    for i in range(cruise_xyz.shape[1]):
        cruise_xyz[0, i, 0] = 10.0 * ((i - 20) * 0.1)

    # --- STATE VARIABLES ---
    # These represent the "Memory" of the onboard computer
    latest_reasoning_lines = ["Initializing System..."]
    frame_counter = 0

    print(f"Starting Async Architecture Simulation...")
    print(f" - Action Update: Every Frame (100% Speed)")
    print(f" - Logic Update:  Every {reasoning_interval} Frames (~10% Speed)")

    try:
        while True:
            # 1. Read ONE frame (Simulating Real-Time Sensor Stream)
            ret, frame = cap.read()
            if not ret: break
            
            # 2. Prepare Inputs
            fr_resized = cv2.resize(frame, (tW, tH))
            fr_rgb = cv2.cvtColor(fr_resized, cv2.COLOR_BGR2RGB)
            
            # Repeat this frame to fill the context window (Simulates "current moment")
            # In a real car, this would be a buffer of the last 3 frames
            sampled_rgb = [fr_rgb] * int(T1*T2)
            tensor = torch.from_numpy(np.stack(sampled_rgb))
            tensor = tensor.permute(0, 3, 1, 2).unsqueeze(0)
            
            messages = helper.create_message(tensor[0])
            instruction_text = "Output the chain-of-thought reasoning of the driving process, then output the future trajectory."
            if isinstance(messages[0]["content"], list):
                messages[0]["content"].append({"type": "text", "text": instruction_text})
            else:
                messages[0]["content"] = instruction_text

            inputs = processor.apply_chat_template(
                messages, tokenize=True, add_generation_prompt=True,
                return_dict=True, return_tensors="pt",
            )
            
            model_inputs = {
                "tokenized_data": inputs,
                "ego_history_xyz": cruise_xyz, 
                "ego_history_rot": cruise_rot,
            }
            model_inputs = helper.to_device(model_inputs, "cuda")

            # --- 3. THE ASYNC LOGIC ---
            t_start = time.time()
            
            # DECISION: Do we run the heavy "Reasoning" engine this frame?
            run_reasoning = (frame_counter % reasoning_interval == 0)
            
            # Set Token Limit based on mode
            # FAST MODE: 1 token (Just enough to trigger trajectory head)
            # SLOW MODE: 64 tokens (Enough to explain the scene)
            current_max_tokens = 64 if run_reasoning else 1

            with torch.autocast("cuda", dtype=torch.bfloat16):
                _, _, extra = model.sample_trajectories_from_data_with_vlm_rollout(
                    data=model_inputs,
                    top_p=0.8, 
                    temperature=0.6, 
                    num_traj_samples=1,
                    max_generation_length=current_max_tokens, 
                    return_extra=True,
                )

            # --- 4. UPDATE STATE ---
            # Trajectory is ALWAYS fresh (calculated every frame)
            # Reasoning is ONLY updated if we ran the heavy engine
            if run_reasoning:
                raw_output = extra.get("cot", [""])[0]
                lines, full_text = clean_and_wrap_text(raw_output, width=65)
                latest_reasoning_lines = lines # Update the "Dashboard"
                status_color = (0, 255, 0) # Green dot = New Thought
            else:
                status_color = (0, 0, 255) # Red dot = Stale/Cached Thought

            # --- 5. VISUALIZATION ---
            # Draw the Dashboard
            banner_height = 60 + (len(latest_reasoning_lines) * 35)
            cv2.rectangle(frame, (0, 0), (W, banner_height), (0, 0, 0), -1)
            
            # Indicator Light (Simulates Async Thread Activity)
            cv2.circle(frame, (W-50, 40), 15, status_color, -1)
            cv2.putText(frame, "CPU", (W-80, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)

            cv2.putText(frame, "ONBOARD REASONING:", (20, 35), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            y = 80
            for line in latest_reasoning_lines:
                cv2.putText(frame, line, (20, y), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                y += 35
            
            out.write(frame)

            # Display Logic
            if frame_counter % 2 == 0: # Update display every other frame to save bandwidth
                _, encoded_img = cv2.imencode('.jpg', frame) 
                clear_output(wait=True)
                display(Image(data=encoded_img))
                fps_val = 1.0 / (time.time() - t_start)
                print(f"⚡ Mode: {'REASONING UPDATE' if run_reasoning else 'ACTION ONLY'}")
                print(f"⚡ Speed: {fps_val:.2f} FPS")

            frame_counter += 1

    finally:
        cap.release()
        out.release()
        print(f"\n✅ Simulation saved to: {out_path}")

# --- Launch ---
def launch_async_dashboard():
    video_files = sorted([f.name for f in VIDEO_DIR.glob("*.mp4")])
    if not video_files: return
    dropdown = widgets.Dropdown(options=video_files, description='Video:', layout=widgets.Layout(width='60%'))
    btn_run = widgets.Button(description='▶ Run Simulation', button_style='warning', icon='cogs')
    out_log = widgets.Output()

    def on_button_click(b):
        with out_log:
            clear_output()
            selected_video = VIDEO_DIR / dropdown.value
            new_name = f"{selected_video.stem}_async_sim.mp4"
            print(f"🚀 Simulating Onboard Computer for: {selected_video.name}...")
            run_async_simulation(
                video_path=selected_video,
                model=model,
                processor=processor,
                tmpl=tmpl,
                out_path=Path(new_name),
                reasoning_interval=10 # Update text every 10 frames
            )

    btn_run.on_click(on_button_click)
    display(widgets.VBox([dropdown, btn_run, out_log]))

launch_async_dashboard()